In [1]:
# !pip3 install --upgrade --quiet  langchain langchain-community langchainhub gpt4all chromadb 
# !pip3 install unstructured
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings import GPT4AllEmbeddings
import os

In [2]:
directories = ["Web Scholar PDFs", "About Scottie", "Buggy News", "academic_calendars", "history_of_cmu", "history_of_scs", "Kiltie Band", "lti_faculty", "lti_programs", "program_handbooks", "Tartan Facts"]

In [4]:
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip install pypdf

!pip3 install ipywidgets


docs = []
for i in directories:
    loader = DirectoryLoader(i, glob="*", exclude="annotation.txt")
    docs.append(loader.load())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.4 MB/s eta 0:00:00a 0:00:01


No /Root object! - Is this really a PDF?
PDF text extraction failed, skip text extraction...
Error loading file Web Scholar PDFs/2af97e7bca7853023ea85e1d4f8eb21a60b4d26b.pdf


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [4]:
all_splits = []
for i in docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    all_splits.append(text_splitter.split_documents(i))

In [5]:
for i in all_splits:
    vectorstore = Chroma.from_documents(documents=i, embedding=GPT4AllEmbeddings(), persist_directory="all_data/")

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: g